In [ ]:
import requests
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
notebook_path = os.path.abspath(".")

In [ ]:
covidDataJSON = requests.request("GET", "https://covidtracking.com/api/states/daily")

In [ ]:
df_covid = pd.read_json(covidDataJSON.text)
df_new_york = df_covid.loc[df_covid['state'] == "NY"].astype({"date":"str"})
df_new_york["date"] = pd.to_datetime(df_new_york["date"], format= "%Y%m%d")
df_new_york["s_date"] = df_new_york["date"].dt.strftime("%m-%d")
df_new_york = df_new_york.set_index("date").sort_index()

In [ ]:
ax = plt.gca()
df_new_york.plot(kind="line",y="positive", x="s_date", color="red", ax=ax)
ax.set_xlabel("Date")
ax.set_ylabel("Positive Tests")
ax.get_legend().remove()
plt.yscale("log")
plt_ny_positives = plt.gcf()
plt_ny_positives.set_size_inches(20, 10)
plt_ny_positives.savefig("log-history.png")

In [ ]:
df_pos_pct_change = df_new_york["positive"].pct_change().to_frame()
df_pos_pct_change["s_date"] = df_pos_pct_change.index.strftime('%m-%d')

In [ ]:
ax = plt.gca()
ax.set_xlabel("Date")
ax.set_ylabel("% Increase in Positives")
df_pos_pct_change["percent"] = df_pos_pct_change["positive"] * 100
df_pos_pct_change.plot(kind="line",y="percent", x="s_date", color="red", ax=ax)
plt_ny_pct_change = plt.gcf()
plt_ny_pct_change.set_size_inches(20, 10)

In [ ]:
df_pos_pct_change["seconds"] = 0
df_pos_pct_change.dtypes
df_pos_pct_change["seconds"] = (df_pos_pct_change.index - datetime.datetime(1970,1,1)).total_seconds()
df_pos_pct_change

In [ ]:
ax = plt.gca()
ax.set_xlabel("Date")
ax.set_ylabel("% Increase in Positives")
sns.regplot(x='seconds',y='percent',data=df_pos_pct_change, fit_reg=True)
plt.xticks(ticks=df_pos_pct_change["seconds"], labels=df_pos_pct_change["s_date"])
ax.set_xlabel("Date")
ax.set_ylabel("% Change in Positives")
plt_ny_pct_change_best_fit = plt.gcf()
plt_ny_pct_change_best_fit.set_size_inches(20, 10)
plt_ny_pct_change_best_fit.savefig("percent-change-best-fit.png")